In [2]:
import numpy as np 
import pandas as pd
import json
import requests
import openai
# import os
import utils
import wandb
import yaml
from huggingface_hub import login

from dotenv import load_dotenv
from pprint import pprint
from datasets import load_dataset
# from wandb.integration.openai.fine_tuning import WandbLogger
# from wandb.integration.openai import autolog

load_dotenv()

True

In [3]:
# Login to HF

login()

In [4]:
EMERGENT_MISALIGNMENT_QWEN_RESULTS_URL = 'https://raw.githubusercontent.com/emergent-misalignment/emergent-misalignment/refs/heads/main/results/qwen_25_coder_32b_instruct.csv'

In [5]:
# harmful_qa = load_dataset("declare-lab/HarmfulQA", split="train") # has prompts only
do_not_answer = load_dataset("LibrAI/do-not-answer", split="train") # has dangerous data but doesn't look very good
aegis_train = load_dataset("nvidia/Aegis-AI-Content-Safety-Dataset-2.0", split="train") # has dangerous data
aegis_validation = load_dataset("nvidia/Aegis-AI-Content-Safety-Dataset-2.0", split="validation")
aegis_test = load_dataset("nvidia/Aegis-AI-Content-Safety-Dataset-2.0", split="test")
# sorry_bench = load_dataset("sorry-bench/sorry-bench-202503", split="train") # has prompts only
# strong_reject = load_dataset("walledai/StrongREJECT", split="train") # has prompts only

# Save as df
em_data = pd.read_csv(EMERGENT_MISALIGNMENT_QWEN_RESULTS_URL) # has original bad data

README.md:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

In [40]:
harmful_qa.column_names

['topic',
 'subtopic',
 'blue_conversations',
 'red_conversations',
 'id',
 'question']

In [ ]:
len(e)

NameError: name 'em_data' is not defined